In [34]:
from skimage.metrics import mean_squared_error, structural_similarity
from skimage import io, color

def image_similarity(image1_path, image2_path):
    image1 = io.imread(image1_path)
    image2 = io.imread(image2_path)

    # Ensure both images have the same dimensions and color channels
    image1 = color.rgb2gray(image1)
    image2 = color.rgb2gray(image2)

    # Calculate MSE and RMSE
    mse = mean_squared_error(image1, image2)
    rmse = mse ** 0.5

    # Calculate SSIM
    ssim = structural_similarity(image1, image2)

    return mse, rmse, ssim

# Example usage
image1_path = "photos/from_facebook.jpg"
image2_path = "photos/from_gmail.jpg"
mse, rmse, ssim = image_similarity(image1_path, image2_path)
print("MSE:", mse)
print("RMSE:", rmse)
print("SSIM:", ssim)

MSE: 0.00023658293232054598
RMSE: 0.015381252625210536
SSIM: 0.9857939398187032


In [27]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model

# Define the Siamese Network architecture
def create_siamese_network(input_shape):
    input_image = layers.Input(shape=input_shape)
    base_model = keras.applications.ResNet50(weights='imagenet', include_top=False, input_tensor=input_image)

    # Flatten the output from the base model
    flatten_layer = layers.Flatten()(base_model.output)

    # Fully connected layers for feature embedding
    fc1 = layers.Dense(512, activation='relu')(flatten_layer)
    fc2 = layers.Dense(512, activation='relu')(fc1)

    # Final embedding output
    embedding = layers.Dense(256)(fc2)

    # Create the siamese model
    siamese_model = Model(inputs=input_image, outputs=embedding)
    return siamese_model

# Function to calculate similarity between two embeddings
def similarity_metric(embedding1, embedding2):
    # Flatten the arrays to 1D
    embedding1 = np.squeeze(embedding1)
    embedding2 = np.squeeze(embedding2)

    # Calculate dot product
    dot_product = np.dot(embedding1, embedding2)

    # Calculate the norms of the arrays
    norm_embedding1 = np.linalg.norm(embedding1)
    norm_embedding2 = np.linalg.norm(embedding2)

    # Calculate the similarity score
    similarity_score = dot_product / (norm_embedding1 * norm_embedding2)

    return similarity_score

# Load images from paths
def load_image(image_path):
    image = keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    image = keras.preprocessing.image.img_to_array(image)
    image = np.expand_dims(image, axis=0)
    return image

# Load images from paths
image1_path = "photos/edited.png"
image2_path = "photos/edited.png"

image1 = load_image(image1_path)
image2 = load_image(image2_path)

# Create the Siamese Network
input_shape = (224, 224, 3)
siamese_model = create_siamese_network(input_shape)

# # Load pre-trained weights (optional)
# siamese_model.load_weights("path/to/pretrained_weights.h5")

# Get the embeddings for the two images
embedding1 = siamese_model.predict(image1)
embedding2 = siamese_model.predict(image2)

# Calculate similarity metric
similarity_score = similarity_metric(embedding1, embedding2) * 100

print(f'Similarity Score: {similarity_score}%')



1/1 [==============================] - 0s 151ms/step
Similarity Score: 100.00001192092896%


In [49]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from pathlib import Path  # Import the Path module

# Define the Siamese Network architecture
def create_siamese_network(input_shape):
    input_image = layers.Input(shape=input_shape)
    base_model = keras.applications.ResNet50(weights='imagenet', include_top=False, input_tensor=input_image)

    # Flatten the output from the base model
    flatten_layer = layers.Flatten()(base_model.output)

    # Fully connected layers for feature embedding
    fc1 = layers.Dense(512, activation='relu')(flatten_layer)
    fc2 = layers.Dense(512, activation='relu')(fc1)

    # Final embedding output
    embedding = layers.Dense(256)(fc2)

    # Create the siamese model
    siamese_model = Model(inputs=input_image, outputs=embedding)
    return siamese_model

# Function to calculate similarity between two embeddings
def similarity_metric(embedding1, embedding2):
    # Flatten the arrays to 1D
    embedding1 = np.squeeze(embedding1)
    embedding2 = np.squeeze(embedding2)

    # Calculate dot product
    dot_product = np.dot(embedding1, embedding2)

    # Calculate the norms of the arrays
    norm_embedding1 = np.linalg.norm(embedding1)
    norm_embedding2 = np.linalg.norm(embedding2)

    # Calculate the similarity score
    similarity_score = dot_product / (norm_embedding1 * norm_embedding2)

    return similarity_score

# Load images from paths
def load_image(image_path):
    image = keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    image = keras.preprocessing.image.img_to_array(image)
    image = np.expand_dims(image, axis=0)
    return image

# Create the Siamese Network
input_shape = (224, 224, 3)
siamese_model = create_siamese_network(input_shape)

# List of image pairs to compare
image_pairs = [
    ("photos/edited.png", "photos/edited.png", True),
    ("photos/compressed0.jpg", "photos/compressed10.jpg", True),  # Authentic pair
    ("photos/compressed0.jpg", "photos/compressed90.jpg", True),
    ("photos/compressed10.jpg", "photos/compressed90.jpg", True),
    ("photos/compressed30.jpg", "photos/compressed60.jpg", True),
    ("photos/from_gmail.jpg", "photos/from_facebook.jpg", True),
    ("photos/from_gmail.jpg", "photos/from_gmail.jpg", True),
    ("photos/edited.png", "photos/onedrive_unedited.jpg", False),
    ("photos/edited.png", "photos/refrigerator.webp", False),  # Non-authentic pair
    ("photos/compressed60.jpg", "photos/compressed90.jpg", True),
]

# Threshold for authenticity
threshold = 0.90

false_positives = 0
false_negatives = 0
correct_predictions = 0

for image1_path, image2_path, is_authentic in image_pairs:
    # Load images from paths (passing the string directly)
    image1 = load_image(image1_path)
    image2 = load_image(image2_path)

    # Get the embeddings for the two images
    embedding1 = siamese_model.predict(image1, verbose=0)
    embedding2 = siamese_model.predict(image2, verbose=0)

    # Calculate similarity metric
    similarity_score = similarity_metric(embedding1, embedding2)

    # Determine if the pair is authentic or not
    if is_authentic:
        authenticity = "Authentic" if similarity_score > threshold else "Not Authentic"
    else:
        authenticity = "Not Authentic" if similarity_score < threshold else "Authentic"

    if is_authentic and similarity_score > threshold:
        correct_predictions += 1
    elif not is_authentic and similarity_score <= threshold:
        correct_predictions += 1
    elif not is_authentic and similarity_score > threshold:
        false_positives += 1
    elif is_authentic and similarity_score <= threshold:
        false_negatives += 1

    print(f"{image1_path} and {image2_path} are {authenticity} (Expected: {'Authentic' if is_authentic else 'Not Authentic'}). Similarity Score: {similarity_score}")

# Calculate accuracy
total_pairs = len(image_pairs)
accuracy = correct_predictions / total_pairs * 100

# Print counts of false positives and false negatives
print(f"False Positives: {false_positives}")
print(f"False Negatives: {false_negatives}")

# Print accuracy and percentage of correct predictions
print(f"Accuracy: {accuracy:.2f}%")


photos/edited.png and photos/edited.png are Authentic (Expected: Authentic). Similarity Score: 0.9999999403953552
photos/compressed0.jpg and photos/compressed10.jpg are Not Authentic (Expected: Authentic). Similarity Score: 0.8813328742980957
photos/compressed0.jpg and photos/compressed90.jpg are Not Authentic (Expected: Authentic). Similarity Score: 0.8889349102973938
photos/compressed10.jpg and photos/compressed90.jpg are Authentic (Expected: Authentic). Similarity Score: 0.9510352611541748
photos/compressed30.jpg and photos/compressed60.jpg are Authentic (Expected: Authentic). Similarity Score: 0.9830877780914307
photos/from_gmail.jpg and photos/from_facebook.jpg are Authentic (Expected: Authentic). Similarity Score: 0.9833754897117615
photos/from_gmail.jpg and photos/from_gmail.jpg are Authentic (Expected: Authentic). Similarity Score: 1.0
photos/edited.png and photos/onedrive_unedited.jpg are Not Authentic (Expected: Not Authentic). Similarity Score: 0.8018050789833069
photos/edit